<a href="https://colab.research.google.com/github/jooyeonroh/Codestates-Project-2/blob/main/cp2_EDA_%26_ALS_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 14.2 MB/s 


In [1]:
import pandas as pd
import numpy as np
import random
from scipy.sparse import csr_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl

from implicit.als import AlternatingLeastSquares

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/cp2_log_data.csv')
df.drop(columns=['Unnamed: 0','category_id','category_code','user_session'],inplace=True)
df

,event_time,event_type,product_id,brand,price,user_id,division1,division2,division3
0,2019-10-01 00:00:00+00:00,view,44600062,shiseido,35.79,541312140,unknown,unknown,unknown
1,2019-10-01 00:00:00+00:00,view,3900821,aqua,33.20,554748717,appliances,environment,water_heater
2,2019-10-01 00:00:01+00:00,view,17200506,unknown,543.10,519107250,furniture,living_room,sofa
3,2019-10-01 00:00:01+00:00,view,1307067,lenovo,251.74,550050854,computers,notebook,notebook
4,2019-10-01 00:00:04+00:00,view,1004237,apple,1081.98,535871217,electronics,smartphone,smartphone
...,...,...,...,...,...,...,...,...,...
38672585,2019-10-31 23:59:58+00:00,view,2300275,gopro,527.40,537931532,electronics,camera,video
38672586,2019-10-31 23:59:58+00:00,view,10800172,redmond,61.75,527322328,unknown,unknown,unknown
38672587,2019-10-31 23:59:58+00:00,view,5701038,kenwood,128.70,566280422,auto,accessories,player
38672588,2019-10-31 23:59:59+00:00,view,21407424,tissot,689.85,513118352,electronics,clocks,clocks


In [ ]:
# path = "/content/drive/MyDrive/cp2_data.parquet"
# df = pd.read_parquet(path, engine='pyarrow')
# df
### ram을 더 많이 잡아 먹어서 csv로 사용하기로했다.

## 아이디와 카테고리별 클릭 수(view)와 구매 여부 분석
- 카테고리는 중분류(division2)를 기준
    - 이유 대분류는 14개 , 중분류는 58개, 소분류는 125개로 대분류보다는 디테일하지만 소분류보다는 복잡하지 않은 중분류로 선택
- 가설 ) 특정 카테고리에 클릭 수가 많으면 해당 카테고리의 제품을 구매할 것이다.

- 분석결과
    - 특정 카테고리별로 클릭수가 10개 이상인 고객의 수 : 640,447명
    - 구매 이력이 있는 고객의 수 : 333,701명
    - 특정 카테고리별로 10번 이상 클릭 하고, 구매까지 이어진 고객 180,194명
        - 10번 이상 클릭하고 구매까지 이어진 고객은 약 28%


In [4]:
print(df['division1'].nunique())
print(df['division2'].nunique())
print(df['division3'].nunique())

14
58
125


In [5]:
g_df = df.copy()
g_df_view = g_df.loc[g_df['event_type']=='view']
g_df_view = g_df_view.groupby(['user_id','division2']).count().sort_values('price',ascending=False).reset_index()
g_df_view = g_df_view[['user_id','division2','product_id']]
g_df_view = g_df_view.rename(columns={'product_id':'count'})

In [6]:
viewer = g_df_view.loc[g_df_view['count']>10]

In [7]:
viewer

,user_id,division2,count
0,512475445,accessories,5158
1,512365995,smartphone,3587
2,513021392,smartphone,2756
3,512505687,smartphone,2697
4,514649263,carriage,2271
...,...,...,...
846328,543186928,unknown,11
846329,542894619,smartphone,11
846330,542979795,unknown,11
846331,542638205,camera,11


In [12]:
# 특정 카테고리의 아이템을 10번 이상 본 고객이 구매까지 이어진 케이스
intersec = viewer.loc[(viewer['user_id'].isin(buyer)&(viewer['division2'].isin(list(buyer_df['division2']))))]
intersec

,user_id,division2,count
2,513021392,smartphone,2756
12,555397664,clocks,1417
15,512437914,shoes,1333
18,549029080,smartphone,1276
19,550860632,clocks,1261
...,...,...,...
846309,543181551,unknown,11
846311,524065451,smartphone,11
846312,521939216,unknown,11
846315,559386629,tools,11


In [13]:
viewer_list = list(viewer['user_id'])

intersec_list = list(set(viewer_list) & set(buyer))
print('n_viewer_list : ', len(set(viewer_list)),
      '\nn_buyer_list : ', len(set(buyer)),
      '\nn_intersec_list : ', len(set(intersec['user_id'])))

n_viewer_list :  640447 
n_buyer_list :  333701 
n_intersec_list :  180194


## 고객 분류
- 기업에 이익을 주느냐의 여부에 의한 고객 분류
    - 잠재고객 : 회사에 인지하고 있지 않거나, 인지하고 있어도 관심이 없는 고객
    - 가망고객 : 회사에 인지 O , 어느 정도 관심 O
    - 신규고객 : 회사와 거래를 시작하는 단계의 고객
    - 기존고개 : 회사와 지속적인 거래
    - 충성고객 : 자신이 뭔가를 구매하려고 마음먹었을 때 그 기업을 먼저 떠올리는 고객, 때로는 입소문도

- 문제 1
    - 한 달 간의 로그데이터를 사용하는 프로젝트로, 한 달 간의 거래가 적거나 많다고해서 고객을 정확히 분류하기 어려움
- 해결방법 1
    - 비록 현재는 한 달 간의 데이터를 사용하지만 추후 더 많은 기간의 데이터를 넣는다고 가정하에 함수 처리 해야함.
    - 고객을 5가지로 나누지 않고 2가지로 나누기
        - **관심고객(잠재고객+가망고객+신규고객)**
        - **단골고객(기존고객+충성고객)**



### 고객당 평균적으로 사는 횟수로 고객 분류
- 실제 구매 경험이 있는 고객 : 333,701 명
- 제일 적은 구매 횟수 : 1개
- 제일 많은 구매 횟수 : 322개
- 평균 : 2개

### 고객 분류 (구매 이력이 있는 고객 기준)
- 관심고객 : 1개 이하 구매 고객 62% ( 209,273 명)
- 단골고객 : 2개 이상 구매 고객 38% ( 124,428 명)

In [12]:
# 구매 경험이 있는 고객
buyer = df.loc[df['event_type']=='purchase']
buyer

,event_time,event_type,product_id,brand,price,user_id,division1,division2,division3
150,2019-10-01 00:02:14+00:00,purchase,1004856,samsung,130.76,543272936,electronics,smartphone,smartphone
286,2019-10-01 00:04:37+00:00,purchase,1002532,apple,642.69,551377651,electronics,smartphone,smartphone
348,2019-10-01 00:06:02+00:00,purchase,5100816,xiaomi,29.51,514591159,unknown,unknown,unknown
405,2019-10-01 00:07:07+00:00,purchase,13800054,santeri,54.42,555332717,furniture,bathroom,toilet
515,2019-10-01 00:09:26+00:00,purchase,4804055,apple,189.91,524601178,electronics,audio,headphone
...,...,...,...,...,...,...,...,...,...
38671860,2019-10-31 23:53:53+00:00,purchase,1004767,samsung,242.63,542774966,electronics,smartphone,smartphone
38672049,2019-10-31 23:55:21+00:00,purchase,47500017,puckator,20.59,514622109,construction,tools,light
38672135,2019-10-31 23:56:03+00:00,purchase,1003306,apple,577.89,512717356,electronics,smartphone,smartphone
38672220,2019-10-31 23:56:53+00:00,purchase,1004240,apple,1054.60,533892594,electronics,smartphone,smartphone


In [5]:
# 구매 경험이 있는 고객 대상으로 평균 횟수구하기
count_buyer = buyer.groupby('user_id').count()[['product_id']].reset_index()
print(count_buyer['product_id'].describe())
count_buyer

count    333701.000000
mean          2.111055
std           3.584361
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         322.000000
Name: product_id, dtype: float64


,user_id,product_id
0,264649825,2
1,303160429,1
2,340041246,4
3,384989212,1
4,396222093,1
...,...,...
333696,566270177,1
333697,566272569,2
333698,566274637,1
333699,566276996,1


In [6]:
count_buyer['product_id'].value_counts(normalize=True)[:5]
# 1개 산 고객이 62%

1    0.627127
2    0.187248
3    0.073140
4    0.037114
5    0.020785
Name: product_id, dtype: float64

In [7]:
# 관심 고객 리스트 만들기
att_buyer_list = list(count_buyer.loc[count_buyer['product_id']==1]['user_id'])

### 고객의  매출 금액
- 관심 고객
    - 평균 : $ 270
    - 총 : $ 56,560,230.4


- 단골 고객 
    - 평균 : $ 1,350 
    - 총 : $ 168,020,561.94

> 확실히 단골 고객을 기준으로 했을 때 고객의 수가 적음에도 평균 매출에 차이가 크다.


In [46]:
# 아이디별 매출액 합계
grouped_buyer = buyer.groupby(['user_id']).sum().reset_index()
grouped_buyer.drop(columns='product_id', inplace=True)
print(grouped_buyer['price'].describe())
grouped_buyer

count    333701.000000
mean        673.000058
std        2101.621661
min           0.880000
25%         111.820000
50%         249.860000
75%         613.920000
max      265569.520000
Name: price, dtype: float64


,user_id,price
0,264649825,1240.04
1,303160429,340.59
2,340041246,915.52
3,384989212,41.44
4,396222093,48.14
...,...,...
333696,566270177,75.94
333697,566272569,254.84
333698,566274637,2011.63
333699,566276996,74.39


In [57]:
#관심 고객 - 아이디별 매출액
grouped_att_buyer = grouped_buyer.loc[grouped_buyer['user_id'].isin(att_buyer_list)]
print(grouped_att_buyer['price'].describe())
print("sum : ", grouped_att_buyer['price'].sum())
grouped_att_buyer

count    209273.000000
mean        270.270080
std         317.079653
min           0.880000
25%          71.820000
50%         161.870000
75%         307.860000
max        2574.040000
Name: price, dtype: float64
sum :  56560230.4


,user_id,price
1,303160429,340.59
3,384989212,41.44
4,396222093,48.14
5,401021311,231.64
6,401646272,76.45
...,...,...
333693,566267125,14.13
333696,566270177,75.94
333698,566274637,2011.63
333699,566276996,74.39


In [58]:
# 단골 고객 - 아이디별 매출액 합계
grouped_reg_buyer = grouped_buyer.loc[~grouped_buyer['user_id'].isin(att_buyer_list)]
print(grouped_reg_buyer['price'].describe())
print("sum : ", grouped_reg_buyer['price'].sum())
grouped_reg_buyer

count    124428.000000
mean       1350.343668
std        3308.283883
min           2.000000
25%         273.487500
50%         578.190000
75%        1392.572500
max      265569.520000
Name: price, dtype: float64
sum :  168020561.94


,user_id,price
0,264649825,1240.04
2,340041246,915.52
7,403013066,807.16
10,415873351,263.06
17,427391662,430.46
...,...,...
333679,566253999,378.39
333684,566256174,2700.54
333694,566267483,30.74
333695,566270060,147.42


## **최종 : 타겟 나눠서 추천시스템 적용하기**

1. 로그데이터 20개 미만이고 구매 이력 없음 (10,480,777개, 27.1%)
2. 로그데이터 20개 이상이고 구매 이력 없음 (15,528,272개, 40.2%)
3. 로그데이터 관계없이, 구매이력 있는 회원 (12,663,541개, 32.7%)


> 총 로그데이터 개수 38,672,590 

In [4]:
# 구매이력이 있는 고객
buyer_df = df.loc[df['event_type']=='purchase']
buyer = list(buyer_df['user_id'].unique())
len(buyer)

333701

In [10]:
buyer_df

,event_time,event_type,product_id,brand,price,user_id,division1,division2,division3
150,2019-10-01 00:02:14+00:00,purchase,1004856,samsung,130.76,543272936,electronics,smartphone,smartphone
286,2019-10-01 00:04:37+00:00,purchase,1002532,apple,642.69,551377651,electronics,smartphone,smartphone
348,2019-10-01 00:06:02+00:00,purchase,5100816,xiaomi,29.51,514591159,unknown,unknown,unknown
405,2019-10-01 00:07:07+00:00,purchase,13800054,santeri,54.42,555332717,furniture,bathroom,toilet
515,2019-10-01 00:09:26+00:00,purchase,4804055,apple,189.91,524601178,electronics,audio,headphone
...,...,...,...,...,...,...,...,...,...
38671860,2019-10-31 23:53:53+00:00,purchase,1004767,samsung,242.63,542774966,electronics,smartphone,smartphone
38672049,2019-10-31 23:55:21+00:00,purchase,47500017,puckator,20.59,514622109,construction,tools,light
38672135,2019-10-31 23:56:03+00:00,purchase,1003306,apple,577.89,512717356,electronics,smartphone,smartphone
38672220,2019-10-31 23:56:53+00:00,purchase,1004240,apple,1054.60,533892594,electronics,smartphone,smartphone


### 1) 20개 미만의 로그데이터가 있는 잠재고객

In [14]:
# 20개 미만의 로그데이터 추출
def prospect(data,n):
    min_id = data['user_id'].value_counts() < n
    min_id = min_id[min_id].index.to_list()
    data = data[data['user_id'].isin(min_id)]
    data = data[~data['user_id'].isin(buyer)]
    return data

In [15]:
pros_user = prospect(df,20)
pros_user

,event_time,event_type,product_id,brand,price,user_id,division1,division2,division3
1,2019-10-01 00:00:00+00:00,view,3900821,aqua,33.20,554748717,appliances,environment,water_heater
2,2019-10-01 00:00:01+00:00,view,17200506,unknown,543.10,519107250,furniture,living_room,sofa
6,2019-10-01 00:00:08+00:00,view,17300353,creed,380.96,555447699,unknown,unknown,unknown
7,2019-10-01 00:00:08+00:00,view,31500053,luminarc,41.16,550978835,unknown,unknown,unknown
8,2019-10-01 00:00:10+00:00,view,28719074,baden,102.71,520571932,apparel,shoes,keds
...,...,...,...,...,...,...,...,...,...
38672572,2019-10-31 23:59:53+00:00,view,1004708,huawei,151.99,566031393,electronics,smartphone,smartphone
38672578,2019-10-31 23:59:56+00:00,view,1003936,xiaomi,437.33,520088904,electronics,smartphone,smartphone
38672581,2019-10-31 23:59:58+00:00,view,12800151,sony,8.49,544578298,unknown,unknown,unknown
38672583,2019-10-31 23:59:58+00:00,view,1004870,samsung,275.25,518956209,electronics,smartphone,smartphone


### 2) 로그데이터 20개 이상이고, 구매기록이 없는 회원

In [32]:
#2) 로그데이터 20개 이상이고, 구매기록이 없는 회원
def wo_purchase(data,n):
    min_id = data['user_id'].value_counts() >= n
    min_id = min_id[min_id].index.to_list()
    data = data[data['user_id'].isin(min_id)]
    data = data[~data['user_id'].isin(buyer)]
    return data

In [17]:
# 20개 이상 & 구매기록 없음
exist = wo_purchase(df,20)
exist

,event_time,event_type,product_id,brand,price,user_id,division1,division2,division3
3,2019-10-01 00:00:01+00:00,view,1307067,lenovo,251.74,550050854,computers,notebook,notebook
4,2019-10-01 00:00:04+00:00,view,1004237,apple,1081.98,535871217,electronics,smartphone,smartphone
12,2019-10-01 00:00:13+00:00,view,3900746,haier,102.38,555444559,appliances,environment,water_heater
17,2019-10-01 00:00:19+00:00,view,1306631,hp,580.89,550050854,computers,notebook,notebook
18,2019-10-01 00:00:19+00:00,view,1005135,apple,1747.79,535871217,electronics,smartphone,smartphone
...,...,...,...,...,...,...,...,...,...
38672584,2019-10-31 23:59:58+00:00,view,2702331,lg,527.43,524356542,appliances,kitchen,refrigerators
38672585,2019-10-31 23:59:58+00:00,view,2300275,gopro,527.40,537931532,electronics,camera,video
38672586,2019-10-31 23:59:58+00:00,view,10800172,redmond,61.75,527322328,unknown,unknown,unknown
38672588,2019-10-31 23:59:59+00:00,view,21407424,tissot,689.85,513118352,electronics,clocks,clocks


### 3) 로그데이터 상관없이, 구매기록이 있는 회원

In [5]:
#3) 로그데이터 상관없이, 구매기록이 있는 회원
def with_purchase(data):
    # min_id = data['user_id'].value_counts() >= n
    # min_id = min_id[min_id].index.to_list()
    # data = data[data['user_id'].isin(min_id)]
    data = data[data['user_id'].isin(buyer)]
    return data

In [6]:
w_purchase = with_purchase(df)
w_purchase

,event_time,event_type,product_id,brand,price,user_id,division1,division2,division3
0,2019-10-01 00:00:00+00:00,view,44600062,shiseido,35.79,541312140,unknown,unknown,unknown
5,2019-10-01 00:00:05+00:00,view,1480613,pulser,908.62,512742880,computers,desktop,desktop
13,2019-10-01 00:00:15+00:00,view,44600062,shiseido,35.79,541312140,unknown,unknown,unknown
21,2019-10-01 00:00:22+00:00,view,1480714,pulser,921.49,512742880,computers,desktop,desktop
22,2019-10-01 00:00:23+00:00,view,1004739,xiaomi,197.55,519530528,electronics,smartphone,smartphone
...,...,...,...,...,...,...,...,...,...
38672562,2019-10-31 23:59:50+00:00,view,13104838,remain,207.98,525530673,unknown,unknown,unknown
38672563,2019-10-31 23:59:51+00:00,view,1005014,samsung,503.09,533326659,electronics,smartphone,smartphone
38672568,2019-10-31 23:59:52+00:00,view,1002524,apple,531.26,565404816,electronics,smartphone,smartphone
38672569,2019-10-31 23:59:53+00:00,view,17200505,unknown,543.10,557880508,furniture,living_room,sofa


### ALS - 추천 시스템 적용 --> 구매없으며 로그데이터 20개 이상
- imp_heat_rate =  0.53
- imp_precision =  0.08
- imp_recall =  0.14

In [22]:
item_lookup1 = exist[['product_id','division1','division2','brand']].drop_duplicates()
item_lookup1

,product_id,division1,division2,brand
3,1307067,computers,notebook,lenovo
4,1004237,electronics,smartphone,apple
12,3900746,appliances,environment,haier
17,1306631,computers,notebook,hp
18,1005135,electronics,smartphone,apple
...,...,...,...,...
38664142,44600230,unknown,unknown,farmstay
38664963,24100818,unknown,unknown,guerisson
38667236,27201900,unknown,unknown,kenzo
38671236,4200837,appliances,environment,mdv


In [21]:
# 1. user_id, product_id
user_product1 = exist[['user_id','product_id']]

# 2. user 별로 product 얼마나 구매,카드,보기 ?
group_user_product1 = user_product1.groupby(['user_id','product_id']).size().to_frame('count').reset_index()
group_user_product1

,user_id,product_id,count
0,209714031,5500007,4
1,209714031,5500032,2
2,209714031,5500097,1
3,209714031,5500107,2
4,209714031,5500108,2
...,...,...,...
8407805,566274665,12709942,5
8407806,566274665,12711208,2
8407807,566274665,12711949,5
8407808,566274665,12711989,1


In [26]:
# test 데이터 30%/ train epdlxj 65% 로 나누기
valid_pct = 0.3
group_user_product1['random'] = np.random.random(size=len(group_user_product1))

train_mask1 = group_user_product1['random'] <  (1 - valid_pct)
valid_mask1 = group_user_product1['random'] >= (1 - valid_pct)

df_train1 = group_user_product1[train_mask1].reset_index()
df_valid1 = group_user_product1[valid_mask1].reset_index()

# 사용자 로그 수에 대한 가중치 저장
sample_weight_train1 = np.log2(df_train1['count'] + 1)
sample_weight_valid1 = np.log2(df_valid1['count'] + 1)
# 필요한 데이터만 남기기
df_train1 = df_train1[['user_id', 'product_id']]
df_valid1 = df_valid1[['user_id', 'product_id']]

# 유니크한 user 행렬
train_users1 = np.sort(df_train1.user_id.unique())
valid_users1 = np.sort(df_valid1.user_id.unique())
cold_start_users1 = set(valid_users1) - set(train_users1)

# 유니크한 product 행렬
train_items1 = np.sort(df_train1.product_id.unique())
valid_items1 = np.sort(df_valid1.product_id.unique())
cold_start_items1 = set(valid_items1) - set(train_items1)

# lookup 테이블도 train / valid 데이터대로 나누기
train_item_lookup1 = item_lookup1[item_lookup1.product_id.isin(train_items1)]
test_item_lookup1 = item_lookup1[item_lookup1.product_id.isin(valid_items1)]

# create zero-based index position <-> user/item ID mappings
index_to_user1 = pd.Series(np.sort(train_users1))
index_to_item1 = pd.Series(np.sort(train_items1))

# create reverse mappings from user/item ID to index positions
user_to_index1 = pd.Series(data=index_to_user1.index, index=index_to_user1.values)
item_to_index1 = pd.Series(data=index_to_item1.index, index=index_to_item1.values)

# convert user/item identifiers to index positions - 유니크한 행렬 만들기
interactions_train_imp1 = df_train1.copy()
interactions_train_imp1['user_id'] = df_train1['user_id'].map(user_to_index1)
interactions_train_imp1['product_id'] = df_train1['product_id'].map(item_to_index1)

# prepare the data for CSR creation
data1 = sample_weight_train1
rows1 = interactions_train_imp1['user_id']
cols1 = interactions_train_imp1['product_id']

# create the required user-item and item-user CSR matrices
items_user_imp1 = csr_matrix((data1, (cols1, rows1)), shape=(len(train_items1), len(train_users1)))
user_items_imp1 = items_user_imp1.T.tocsr()

In [27]:
# initialize and fit the model
imp_model1 = AlternatingLeastSquares(factors=64, 
                                    alpha = 30,
                                    random_state=42,
                                    regularization=0.05,
                                    iterations=20, 
                                    calculate_training_loss=True)
imp_model1.fit(user_items_imp1)

  0%|          | 0/20 [00:00<?, ?it/s]

In [30]:
# generate recommendations for all users and map back to original user/item ID values
recs_imp1 = imp_model1.recommend_all(user_items=user_items_imp1, N=10, filter_already_liked_items=False)
recs_imp1 = pd.DataFrame(recs_imp1, index=index_to_user1.values).apply(lambda c: c.map(index_to_item1))

valid_user_items1 = df_valid1.groupby('user_id')['product_id'].apply(set).to_dict()
combined_users1 = set(train_users1) & set(valid_users1)

imp_hrt1 = np.mean([int(len(set(recs_imp1.loc[u]) & valid_user_items1[u]) > 0) for u in combined_users1])
imp_pre1 = np.mean([len(set(recs_imp1.loc[u]) & valid_user_items1[u]) / len(recs_imp1.loc[u]) for u in combined_users1])
imp_rec1 = np.mean([len(set(recs_imp1.loc[u]) & valid_user_items1[u]) / len(valid_user_items1[u]) for u in combined_users1])

print('imp_hrt = ', round(imp_hrt1,2))
print('imp_pre = ', round(imp_pre1,2))
print('imp_rec = ', round(imp_rec1,2))

imp_hrt =  0.53
imp_pre =  0.08
imp_rec =  0.14


### ALS - 추천 시스템 적용 --> 구매 이력이 있는 고객의 로그데이터

- imp_heat_rate = 0.53
- imp_precision = 0.08
- imp_recall = 0.14 



In [7]:
item_lookup2 = w_purchase[['product_id','division1','division2','brand']].drop_duplicates()
item_lookup2

,product_id,division1,division2,brand
0,44600062,unknown,unknown,shiseido
5,1480613,computers,desktop,pulser
21,1480714,computers,desktop,pulser
22,1004739,electronics,smartphone,xiaomi
26,27500014,unknown,unknown,redmond
...,...,...,...,...
38667025,49300168,unknown,unknown,persona
38667286,49300142,unknown,unknown,weekend
38668232,60000013,unknown,unknown,norfin
38669155,2402678,appliances,kitchen,teka


In [8]:
# 1. user_id, product_id
user_product2 = w_purchase[['user_id','product_id']]

# 2. user 별로 product 얼마나 구매,카드,보기 ?
group_user_product2 = user_product2.groupby(['user_id','product_id']).size().to_frame('count').reset_index()
group_user_product2

# test 데이터 30%/ train epdlxj 65% 로 나누기
valid_pct = 0.3
group_user_product2['random'] = np.random.random(size=len(group_user_product2))

train_mask2 = group_user_product2['random'] <  (1 - valid_pct)
valid_mask2 = group_user_product2['random'] >= (1 - valid_pct)

df_train2 = group_user_product2[train_mask2].reset_index()
df_valid2 = group_user_product2[valid_mask2].reset_index()

# 사용자 로그 수에 대한 가중치 저장
sample_weight_train2 = np.log2(df_train2['count'] + 1)
sample_weight_valid2 = np.log2(df_valid2['count'] + 1)
# 필요한 데이터만 남기기
df_train2 = df_train2[['user_id', 'product_id']]
df_valid2 = df_valid2[['user_id', 'product_id']]

# 유니크한 user 행렬
train_users2 = np.sort(df_train2.user_id.unique())
valid_users2 = np.sort(df_valid2.user_id.unique())
cold_start_users2 = set(valid_users2) - set(train_users2)

# 유니크한 product 행렬
train_items2 = np.sort(df_train2.product_id.unique())
valid_items2 = np.sort(df_valid2.product_id.unique())
cold_start_items2 = set(valid_items2) - set(train_items2)

# lookup 테이블도 train / valid 데이터대로 나누기
train_item_lookup2 = item_lookup2[item_lookup2.product_id.isin(train_items2)]
test_item_lookup2 = item_lookup2[item_lookup2.product_id.isin(valid_items2)]

# create zero-based index position <-> user/item ID mappings
index_to_user2 = pd.Series(np.sort(train_users2))
index_to_item2 = pd.Series(np.sort(train_items2))

# create reverse mappings from user/item ID to index positions
user_to_index2 = pd.Series(data=index_to_user2.index, index=index_to_user2.values)
item_to_index2 = pd.Series(data=index_to_item2.index, index=index_to_item2.values)

# convert user/item identifiers to index positions - 유니크한 행렬 만들기
interactions_train_imp2 = df_train2.copy()
interactions_train_imp2['user_id'] = df_train2['user_id'].map(user_to_index2)
interactions_train_imp2['product_id'] = df_train2['product_id'].map(item_to_index2)

# prepare the data for CSR creation
data2 = sample_weight_train2
rows2 = interactions_train_imp2['user_id']
cols2 = interactions_train_imp2['product_id']

# create the required user-item and item-user CSR matrices
items_user_imp2 = csr_matrix((data2, (cols2, rows2)), shape=(len(train_items2), len(train_users2)))
user_items_imp2 = items_user_imp2.T.tocsr()

In [13]:
# initialize and fit the model
imp_model2 = AlternatingLeastSquares(factors=64, 
                                    alpha = 30,
                                    random_state=42,
                                    regularization=0.05,
                                    iterations=20, 
                                    calculate_training_loss=True)
imp_model2.fit(user_items_imp2)

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:29: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


  0%|          | 0/20 [00:00<?, ?it/s]

In [14]:
# generate recommendations for all users and map back to original user/item ID values
recs_imp2 = imp_model2.recommend_all(user_items=user_items_imp2, N=10, filter_already_liked_items=False)
recs_imp2 = pd.DataFrame(recs_imp2, index=index_to_user2.values).apply(lambda c: c.map(index_to_item2))

valid_user_items2 = df_valid2.groupby('user_id')['product_id'].apply(set).to_dict()
combined_users2 = set(train_users2) & set(valid_users2)

imp_hrt2 = np.mean([int(len(set(recs_imp2.loc[u]) & valid_user_items2[u]) > 0) for u in combined_users2])
imp_pre2 = np.mean([len(set(recs_imp2.loc[u]) & valid_user_items2[u]) / len(recs_imp2.loc[u]) for u in combined_users2])
imp_rec2 = np.mean([len(set(recs_imp2.loc[u]) & valid_user_items2[u]) / len(valid_user_items2[u]) for u in combined_users2])

print('imp_hrt = ', round(imp_hrt2,2))
print('imp_pre = ', round(imp_pre2,2))
print('imp_rec = ', round(imp_rec2,2))

imp_hrt =  0.49
imp_pre =  0.07
imp_rec =  0.2


참고 : https://medium.com/towards-data-science/factorization-machines-for-item-recommendation-with-implicit-feedback-data-5655a7c749db


### 실제 추천 시스템 적용 test

In [ ]:
user_id = 523974502

user_prod_freq1 = (df.loc[df["user_id"] == user_id, "product_id"].value_counts().iloc[:10])

freq_item1 = item_lookup1.set_index("product_id").loc[user_prod_freq1.index.to_list(), :]
freq_item1.loc[:, "counts"] = user_prod_freq1.to_list()
freq_item1 = freq_item1.reset_index()
freq_item1


,product_id,division1,division2,brand,counts
0,5100563,electronics,clocks,samsung,775
1,5100718,electronics,clocks,samsung,33
2,5100737,electronics,clocks,samsung,20
3,1004249,electronics,smartphone,apple,18
4,5100862,electronics,clocks,samsung,13
5,1005115,electronics,smartphone,apple,10
6,5100738,electronics,clocks,samsung,8
7,1005144,electronics,smartphone,apple,5
8,1004250,electronics,smartphone,apple,5
9,4803977,electronics,audio,samsung,5


In [ ]:
user_idx1 = list(np.where(train_users == user_id)[0])[0]

ids1, scores1 = imp_model1.recommend(user_idx1, user_items_imp1[user_idx1], filter_already_liked_items=False)

als_rec1 = pd.DataFrame({"product_id": df_train["product_id"][ids1], 
                        "score": scores1, 
                        "already_carted": np.in1d(ids1, user_items_imp1[user_idx1].indices)})
als_rec1.drop_duplicates("product_id")


383842

In [ ]:
als_item_lookup1 = train_item_lookup1.set_index("product_id").loc[als_rec1["product_id"].to_list(), :].reset_index()
als_item_lookup1.merge(als_rec1).drop_duplicates("product_id").reset_index(drop=True)

,product_id,division1,division2,brand,score,already_carted
0,5700311,auto,accessories,alpine,1.148470,False
1,5700235,auto,accessories,alpine,1.147167,False
2,12708937,unknown,unknown,michelin,1.129361,False
3,3100971,appliances,kitchen,moulinex,1.054162,False
4,1002524,electronics,smartphone,apple,1.046295,True
5,1005105,electronics,smartphone,apple,1.035718,True
6,15201311,unknown,unknown,bosch,1.019471,True
7,30000203,construction,tools,redbo,1.016864,True
8,25503495,unknown,unknown,febest,1.010126,True
9,12704579,unknown,unknown,nokian,1.009782,False
